In [31]:
import os
import shutil
import glob


def train_test_val_split(size, train_split, test_split, src_dir):
    category = ["normal", "pneumonia"]
    split_list = ["train", "test", "val"]
    for split in split_list:
        if not os.path.exists("Project_Day\\" + split + "\\" ): 
            os.mkdir("Project_Day\\" + split + "\\")
        for cat in category:
            if not os.path.exists("Project_Day\\" + split + "\\" + cat + "\\"): 
                os.mkdir("Project_Day\\" + split + "\\" + cat + "\\")
   
    index = 1
    
    for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
        if (index<=(size*train_split)):
            if "normal" in jpgfile:
                shutil.copy(jpgfile, "Project_Day\\" + "train\\" + "normal\\")

            else:
                shutil.copy(jpgfile, "Project_Day\\" + "train\\" + "pneumonia\\")

            index+=1

        elif ((size*train_split)<index<=((size*train_split)+ (size*test_split))):
            if "normal" in jpgfile:
                shutil.copy(jpgfile, "Project_Day\\" + "test\\" + "normal\\")

            else:
                shutil.copy(jpgfile, "Project_Day\\" + "test\\" + "pneumonia\\")

            index+=1
        
        else:
            if "normal" in jpgfile:
                shutil.copy(jpgfile, "Project_Day\\" + "val\\" + "normal\\")

            else:
                shutil.copy(jpgfile, "Project_Day\\" + "val\\" + "pneumonia\\")


In [32]:
train_test_val_split(2200, 0.8, 0.1, "Project_Day\\data")

In [35]:
index = 0
for file in os.listdir("Project_Day\\val\\pneumonia"):
    index+=1
print(index)

110


In [1]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'Project_Day/train',
        target_size=(256, 256),
        batch_size=32,
        class_mode='binary')
        
validation_generator = val_datagen.flow_from_directory(
        'Project_Day/val',
        target_size=(256, 256),
        batch_size=32,
        class_mode='binary')

Found 1760 images belonging to 2 classes.
Found 220 images belonging to 2 classes.


In [5]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D
from keras.models import Model
from keras import Input
from keras.utils.vis_utils import plot_model
import visualkeras 

input_layer = Input(shape=(256,256,3), name='input_layer')
conv_1 = Conv2D(64, kernel_size=(5,5))(input_layer)
conv_2 = Conv2D(32,kernel_size=(3,3))(conv_1)
pool_1 = MaxPooling2D(pool_size=(2,2), strides=(1,1))(conv_2)
conv_3 = Conv2D(32, kernel_size=(3,3))(pool_1)
flatten = Flatten()(conv_3)
dropout = Dropout(0.3)(flatten)
dense_1 = Dense(64, activation="relu")(dropout)
dense_2 = Dense(32, activation="relu")(dense_1)
output_layer = Dense(1, activation="sigmoid")(dense_2)
model = Model(input_layer, output_layer)

dot_img_file = 'model_2.png'
plot_model(model, to_file=dot_img_file, show_shapes=True)
model.summary()
#visualkeras.layered_view(model) 

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_3 (Conv2D)           (None, 252, 252, 64)      4864      
                                                                 
 conv2d_4 (Conv2D)           (None, 250, 250, 32)      18464     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 249, 249, 32)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 247, 247, 32)      9248      
                                                                 
 flatten_1 (Flatten)         (None, 1952288)           0         
                                                           

In [9]:
from keras.losses import BinaryCrossentropy
batch_size = 32
epochs = 15
steps_per_epochs=128

model.compile(loss=BinaryCrossentropy(), optimizer="adam", metrics=["accuracy"])

In [8]:
history = model.fit(train_generator, steps_per_epoch=steps_per_epochs, epochs=epochs, validation_data=validation_generator, validation_steps=800)

Epoch 1/15
  23/2000 [..............................] - ETA: 3:38:56 - loss: 69.5722 - accuracy: 0.4891

KeyboardInterrupt: 